In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from setup import *

%load_ext autoreload
%autoreload 2

In [2]:
hh = pd.read_csv(survey_dir+"household.csv")
pr = pd.read_csv(survey_dir+"person.csv")
pl = pd.read_csv(survey_dir+"place.csv")
pt = pd.read_csv(survey_dir+"place_transit.csv")
lc = pd.read_csv(survey_dir+"location.csv")

/home/jtl/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (45,50) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jtl/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (11,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
num_pl = len(pl)
print(len(hh), "Households", len(pr), "People", len(pl), "Places", len(pl[pl.placeno!=1]), "Trips")

12391 Households 30683 People 128229 Places 99652 Trips


## Filter Invalid Records

In [4]:
# Income
f_inc = (hh.hhinc >= 0) | (hh.hhinc2 >= -1)
print(np.sum(f_inc==0), 'households with no income')

30 households with no income


In [5]:
# Trip numbers
f_hhtrips = (hh.hhtrips>0)
print(np.sum(f_hhtrips==0), 'households with no trips')

349 households with no trips


In [6]:
hh = hh[f_inc & f_hhtrips]

In [7]:
# Age
f_age = (pr.age >= 12)
print(np.sum(f_age==0), 'people with no age or younger than 12 years')

4959 people with no age or younger than 12 years


In [8]:
# License
f_lic = (pr.lic >= -1)
print(np.sum(f_lic==0), 'people with no license info')

2 people with no license info


In [9]:
# Education
f_edu = (pr.educ > 0)
print(np.sum(f_edu==0), 'people with no education info')

42 people with no education info


In [10]:
pr = pr[f_age & f_lic & f_edu]

In [11]:
# Travel Mode
f_mode = (pl['mode'] >= -1) & (pl['mode'] // 100 < 8)
print(np.sum(f_mode==0), 'places with invalid modes (not available or air travel)')

331 places with invalid modes (not available or air travel)


In [12]:
# Activity Duration
f_actdur = pl['actdur'] < 0

In [13]:
pl = pl[f_mode]

In [14]:
# Trip Location
f_state_loc = (lc['out_region']==0)

pl = pd.merge(lc[f_state_loc][['sampno','locno']], pl, on=['sampno','locno'])

print(np.sum(f_state_loc==0), 'locations not in valid states')
print(num_pl-len(pl), 'places not in valid states')

4945 locations not in valid states
2415 places not in valid states


In [15]:
lc = lc[f_state_loc]

In [16]:
print("Left with", len(hh), "Households", len(pr), "People", len(pl), "Places", len(pl[pl.placeno!=1]), "Trips")

Left with 12013 Households 25698 People 125814 Places 97691 Trips


# Household Features

In [17]:
### INCOME
hh['hh_inc_0_30'] = (hh.hhinc.isin([1,2,3]))|(hh.hhinc2 == 1) # 13%
hh['hh_inc_30_60'] = (hh.hhinc.isin([4,5,6]))|(hh.hhinc2 == 2) # 18% 
hh['hh_inc_60_100'] = (hh.hhinc.isin([7,8]))|(hh.hhinc2 == 3) # 26%
hh['hh_inc_100_150'] = (hh.hhinc.isin([9]))|(hh.hhinc2 == 5) # 23%
hh['hh_inc_150'] = (hh.hhinc == 10)|(hh.hhinc2 == 5) # 19%

# there are no 0s in household size 
hh.hhsize = hh.hhsize * (hh.hhsize > 0)
hh.hhveh = hh.hhveh * (hh.hhveh > 0)
hh['avg_pr_veh'] = hh.hhveh / hh.hhsize

# home ownership (with and without mortgage) (65%)
hh['home_own'] = hh.homeown.isin([0,1,2])
# home type
hh['home_house'] = hh.resty.isin([1,2])
hh['home_condo'] = hh.resty == 3

In [18]:
hh = hh[['sampno','hh_inc_0_30','hh_inc_30_60','hh_inc_60_100','hh_inc_100_150','hh_inc_150',
        'avg_pr_veh','home_own','home_house','home_condo']]
for c in ['sampno','hh_inc_0_30','hh_inc_30_60','hh_inc_60_100','hh_inc_100_150','hh_inc_150',
         'home_own','home_house','home_condo']:
    hh[c] = hh[c].astype(int)

In [19]:
hh.to_csv(data_dir+"household.csv", index=False)

# Person Features
- percentage quoted in terms of # observations (unweighted)

In [19]:
### AGE
pr['12_18yrs'] = (pr['age'] >= 12) & (pr['age'] < 18) # 8%
pr['18_25yrs'] = (pr['age'] >= 18) & (pr['age'] < 25) # 8% 
pr['25_55yrs'] = (pr['age'] >= 25) & (pr['age'] < 55) # 46%
pr['55+yrs'] = (pr['age'] >= 55) # 21%
pr['no_age'] = pr['age'] < 0

### DISABILITY
# 4% the rest is reported not disabled, young people (who do not need to answer), and people who do not wish to disclose
pr['disability'] = pr['disab'] == 1

### EDUCATION
pr['educ_col'] = pr.educ == 5
pr['educ_grad'] = pr.educ == 6

### RACE
# 78%
pr['race_white'] = pr.race == 1 
# 10%
pr['race_black'] = pr.race == 2
# 5%
pr['race_asian'] = pr.race == 3

### SEX
pr['male'] = pr.sex == 1
pr['female'] = pr.sex == 2

### EMPLOYMENT
# employed (58%)
pr['emply'] = pr.emply_ask == 1
# employer-subsidized parking (39%)
pr['emply_park'] = (pr.emply_park > 0) & (pr.emply_park < 6)
# employer-subsidized transit (12%)
pr['emply_transit'] = (pr.emply_transit > 0) & (pr.emply_transit < 5)
# employer provides vehicle (4%)
pr['emply_veh'] = pr.pervh == 1
# days WFH (if not, then 0) (>0: 10%)
pr['emply_wfh'] = pr.tcdays * (pr.tcdays > 0)
# ability to set or change work hours (25%)
pr['emply_flex'] = pr.wkflex == 1
# work hours
pr['emply_hours'] = pr.wrkhrs * (pr.wrkhrs > 0)

### TRAVEL BEHAVIOR
# have license (71%)
pr['license'] = pr.lic == 1
# if skipped, then 0
pr['person_trips'] = pr.pertrips * (pr.pertrips > 0)
# ride bus more than 0 times in a typical week (56%)
pr['person_transit'] = pr.ribus > 0
# frequent bus rider (>=5 times a week) (10%)
pr['person_freq_transit'] = pr.ribus == 4

# 1: Active; 2: Auto; 3: Mobility Services; 4: Public Transit; 5: WFH
# 57% (30,000 valid obervations)
pr['work_mode'] = pr['wmode'].map({1:1,2:1,3:2,4:2,5:2,6:2,7:3,8:4,9:4,10:4,11:3,12:3,14:3,15:3,16:3,18:5})
pr['work_mode'].fillna(0, inplace=True)


In [20]:
pr = pr[['sampno','perno','wtperfin',
         '12_18yrs','18_25yrs','25_55yrs','55+yrs','no_age',
         'disability','educ_col','educ_grad','race_white','race_black','race_asian','male','female',
         'emply_park','emply_transit','emply_veh','emply_wfh','emply_flex','emply_hours',
         'license','person_trips','person_transit','person_freq_transit','work_mode']]
for c in ['12_18yrs','18_25yrs','25_55yrs','55+yrs','no_age',
         'disability','educ_col','educ_grad','race_white','race_black','race_asian','male','female',
         'emply_park','emply_transit','emply_veh','emply_wfh','emply_flex',
         'license','person_trips','person_transit','person_freq_transit','work_mode']:
    pr[c] = pr[c].astype(int)

In [22]:
pr.to_csv(data_dir+"person.csv", index=False)

# Individual Trip Features (Disaggregate)

In [21]:
### Turn Places into Trips
# pl contains the information for the destination (as well as activity duration at destination, trip distance, etc.)
pl['prev_placeGroup'] = pl['placeGroup'] - 1

# create a new pl dataframe for information at the origin
# only need to know where and when 
pl_new = pl[['sampno', 'perno', 'placeGroup', 'locno', 'deptime']].copy()
pl_new = pl_new.sort_values(by=['sampno','perno','placeGroup']).reset_index(drop=True)

tp = pd.merge(pl_new, pl, left_on=['sampno', 'perno', 'placeGroup'], right_on=['sampno', 'perno', 'prev_placeGroup'], suffixes=('_1','_2'))


In [22]:
### Add geospatial info from locations table
lc = lc[['sampno', 'locno', 'loctype', 'state', 'country', 'state_fips',
       'county_fips', 'tract_fips', 'home', 'latitude',
       'longitude']]
tp = pd.merge(tp, lc, left_on=['sampno', 'locno_1'], right_on = ['sampno', 'locno'])
del tp['locno']
tp = pd.merge(tp, lc, left_on=['sampno', 'locno_2'], right_on = ['sampno', 'locno'], suffixes=("_1", "_2"))

In [23]:
# Filter out trips where time and distance reported do not make sense
tp = tp[tp.time_distance_flag==0]

In [24]:
### 
tp['act_dur'] = tp.actdur

tp['arr_hour'] = pd.to_datetime(tp['arrtime']).dt.hour
tp['arr_min'] = pd.to_datetime(tp['arrtime']).dt.minute

tp['dep_hour'] = pd.to_datetime(tp['deptime_1']).dt.hour
tp['dep_min'] = pd.to_datetime(tp['deptime_1']).dt.minute

tp['travel_time'] = tp['travtime']

# excluding the respondent
tp['companion'] = tp.hhparty + tp.nonhhcount
tp['distance'] = tp.distance

### TRIP PURPOSE
# home2 is inferred by survey staff makes up by 1,2 primarily
tp['from_home'] = tp.home_1
tp['to_home'] = tp.home_2 #tpurp.isin([1,2]) # 35%

tp['purp_work'] = tp.tpurp.isin([3,4,5,]) # 18%
tp['purp_school'] = tp.tpurp.isin([6]) # 6%
tp['purp_errand'] = tp.tpurp.isin([8,9,10,11,12,13,14,15,16]) # 19%
tp['purp_recreation'] = tp.tpurp.isin([17,18,19,20,21,22,23,24,25,26,27]) # 21%

tp['ontime_important'] = tp.trip_appt.isin([3,4,5])


In [25]:
tp = tp[['sampno', 'perno', 'placeGroup_1', 'placeGroup_2', 'locno_1', 'locno_2',
         'loctype_1', 'state_1', 'country_1', 'state_fips_1',
         'county_fips_1', 'tract_fips_1', 'latitude_1', 'longitude_1',
         'loctype_2', 'state_2', 'country_2', 'state_fips_2',
         'county_fips_2', 'tract_fips_2', 'latitude_2', 'longitude_2',
         'dep_hour','dep_min','arr_hour','arr_min','travel_time', 'mode', 
         'companion','distance','from_home','to_home',
         'purp_work','purp_school','purp_errand','purp_recreation',
         'ontime_important']]
for c in ['companion','from_home','to_home',
         'purp_work','purp_school','purp_errand','purp_recreation',
         'ontime_important']:
    tp[c] = tp[c].astype(int)

In [26]:
### Merge trip with household and person information
tp = pd.merge(tp, pr, on=['sampno','perno'])
tp = pd.merge(tp, hh, on='sampno')

In [27]:
tp['tract_1'] = tp['state_fips_1'].astype(str)+"_"+tp['county_fips_1'].astype(str)+"_"+tp['tract_fips_1'].astype(str)
tp['tract_2'] = tp['state_fips_2'].astype(str)+"_"+tp['county_fips_2'].astype(str)+"_"+tp['tract_fips_2'].astype(str)

In [28]:
## Simplify Mode Representation
tp['mode'] = tp['mode']//100
tp['mode'] = tp['mode'].map({1:1,2:2,3:3,4:3,5:4,6:3,7:3})
print("1: Active; 2: Auto; 3: Mobility Services; 4: Public Transit")
print(tp.groupby('mode').sum()['wtperfin']/tp['wtperfin'].sum()*100)

1: Active; 2: Auto; 3: Mobility Services; 4: Public Transit
mode
1    12.686272
2    76.148262
3     4.137105
4     7.028362
Name: wtperfin, dtype: float64


In [31]:
print("Exported %d trip observations." % (len(tp)))
print("Columns:", list(tp.columns))
tp.to_csv(data_dir+"trips.csv", index=False)

Exported 84143 trip observations.
Columns: ['sampno', 'perno', 'placeGroup_1', 'placeGroup_2', 'locno_1', 'locno_2', 'loctype_1', 'state_1', 'country_1', 'state_fips_1', 'county_fips_1', 'tract_fips_1', 'latitude_1', 'longitude_1', 'loctype_2', 'state_2', 'country_2', 'state_fips_2', 'county_fips_2', 'tract_fips_2', 'latitude_2', 'longitude_2', 'dep_hour', 'dep_min', 'arr_hour', 'arr_min', 'travel_time', 'mode', 'companion', 'distance', 'from_home', 'to_home', 'purp_work', 'purp_school', 'purp_errand', 'purp_recreation', 'ontime_important', 'wtperfin', '12_18yrs', '18_25yrs', '25_55yrs', '55+yrs', 'no_age', 'disability', 'educ_col', 'educ_grad', 'race_white', 'race_black', 'race_asian', 'male', 'female', 'emply_park', 'emply_transit', 'emply_veh', 'emply_wfh', 'emply_flex', 'emply_hours', 'license', 'person_trips', 'person_transit', 'person_freq_transit', 'work_mode', 'hh_inc_0_30', 'hh_inc_30_60', 'hh_inc_60_100', 'hh_inc_100_150', 'hh_inc_150', 'avg_pr_veh', 'home_own', 'home_house',

# Aggregate

### Mode choice by origin census tract

In [29]:
o_mode = tp.groupby(['state_fips_1', 'county_fips_1', 'tract_fips_1', 'mode']).agg({'wtperfin':sum, 'sampno':'count'}).reset_index()


o_mode['key'] = 0
o_mode_full = pd.merge(o_mode[['state_fips_1', 'county_fips_1', 'tract_fips_1','key']].drop_duplicates(), 
                       o_mode[['mode','key']].drop_duplicates())

o_mode = o_mode.drop("key",1)
o_mode_full = o_mode_full.drop("key",1)

o_mode = pd.merge(o_mode_full, o_mode, on=['state_fips_1', 'county_fips_1', 'tract_fips_1','mode'], how='left').fillna(0)

trip_generation = tp.groupby(['state_fips_1', 'county_fips_1',
       'tract_fips_1']).agg({'wtperfin':sum, 'sampno':'count'}).reset_index()

o_mode = pd.merge(o_mode, trip_generation, on=['state_fips_1', 'county_fips_1', 'tract_fips_1'],
                 suffixes=("_mode","_all"))

o_mode['geoid'] = o_mode['state_fips_1'].astype(str)+'_'+o_mode['county_fips_1'].astype(str)+'_'+o_mode['tract_fips_1'].astype(str)


In [64]:
o_mode.to_csv(data_dir+"origin_trip_behavior.csv", index=False)

In [65]:
print("%d origin census tracts exported." % (len(o_mode)/4))

1998 origin census tracts exported.


### Mode choice by destination census tract

In [30]:
d_mode = tp.groupby(['state_fips_2', 'county_fips_2', 'tract_fips_2', 'mode']).agg({'wtperfin':sum, 'sampno':'count'}).reset_index()
d_mode['key'] = 0
d_mode_full = pd.merge(d_mode[['state_fips_2', 'county_fips_2', 'tract_fips_2','key']].drop_duplicates(), 
                       d_mode[['mode','key']].drop_duplicates())

d_mode = d_mode.drop("key",1)
d_mode_full = d_mode_full.drop("key",1)

d_mode = pd.merge(d_mode_full, d_mode, on=['state_fips_2', 'county_fips_2', 'tract_fips_2','mode'], how='left').fillna(0)

trip_generation = tp.groupby(['state_fips_2', 'county_fips_2',
       'tract_fips_2']).agg({'wtperfin':sum, 'sampno':'count'}).reset_index()

d_mode = pd.merge(d_mode, trip_generation, on=['state_fips_2', 'county_fips_2', 'tract_fips_2'],
                 suffixes=("_mode","_all"))
d_mode['geoid'] = d_mode['state_fips_2'].astype(str)+'_'+d_mode['county_fips_2'].astype(str)+'_'+d_mode['tract_fips_2'].astype(str)


In [70]:
d_mode.to_csv(data_dir+"destination_trip_behavior.csv", index=False)
print("%d destination census tracts exported." % (len(d_mode)/4))

1999 destination census tracts exported.


### Mode Choice by OD pairs

In [34]:
od_weighted = tp.groupby(['tract_1','tract_2'], as_index=False).sum()[['tract_1','tract_2','wtperfin']]
od_count = tp.groupby(['tract_1','tract_2'], as_index=False).count()[['tract_1','tract_2','sampno']]
od = pd.merge(od_weighted, od_count, on=['tract_1','tract_2'])

print("Trips are weighted to reflect representativeness of each sample.")
print("# OD pairs: %d \t # observations: %d \t # weighted trips: %d" %(len(od), od.sum()['sampno'], od.sum()['wtperfin']))
print("# OD pairs with >5 records: %d accounting for %d observations (%d weighted trips)" %\
      (len(od[od['sampno']>5]), od[od['sampno']>5].sum()['sampno'], od[od['sampno']>5].sum()['wtperfin']))

Trips are weighted to reflect representativeness of each sample.
# OD pairs: 44253 	 # observations: 84143 	 # weighted trips: 22363279
# OD pairs with >5 records: 1835 accounting for 23504 observations (5504411 weighted trips)


In [35]:
od_mode = tp.groupby(['tract_1','tract_2','mode'], as_index=False).sum()[['tract_1','tract_2','mode','wtperfin']]

od_mode['key'] = 0
od['key'] = 0

# Create a df with all combinations of od and mode
od_mode_full = pd.merge(od[['tract_1','tract_2','key']].drop_duplicates(), od_mode[['mode','key']].drop_duplicates()).drop("key", 1)
od_mode = od_mode.drop("key",1)
od = od.drop("key",1)

od_mode = pd.merge(od_mode_full, od_mode, on=['tract_1','tract_2','mode'], how='outer').fillna(0)
od_mode = pd.merge(od_mode, od, on=['tract_1','tract_2'], suffixes=("_mode","_od"))
od_mode['mode_share'] = od_mode['wtperfin_mode'] / od_mode['wtperfin_od']

In [36]:
print("Exported full OD mode matrix with %d pairs of ODs." % (len(od)))
od_mode.to_csv(data_dir+"od_mode.csv", index=False)

Exported full OD mode matrix with 44253 pairs of ODs.


- Filter OD pairs to pairs with more than 5 observations
- A lot of public transit trips are filltered. (possibly due to interstate train travel or commuter rail)

In [37]:
od = od[od['sampno']>5]

In [38]:
od_mode = tp.groupby(['tract_1','tract_2','mode'], as_index=False).sum()[['tract_1','tract_2','mode','wtperfin']]

od_mode['key'] = 0
od['key'] = 0
# Use filtered OD (>5 trips)
od_mode_full = pd.merge(od[['tract_1','tract_2','key']].drop_duplicates(), od_mode[['mode','key']].drop_duplicates()).drop("key", 1)
od_mode = od_mode.drop("key",1)
od = od.drop("key",1)

od_mode = pd.merge(od_mode_full, od_mode, on=['tract_1','tract_2','mode'], how='outer').fillna(0)
od_mode = pd.merge(od_mode, od, on=['tract_1','tract_2'], suffixes=("_mode","_od"))
od_mode['mode_share'] = od_mode['wtperfin_mode'] / od_mode['wtperfin_od']

In [39]:
print("Exported filtered OD mode matrix with %d pairs of ODs." % (len(od)))
od_mode.to_csv(data_dir+"od_mode_filtered.csv", index=False)

Exported filtered OD mode matrix with 1835 pairs of ODs.


# Train Test Split of Census Tracts

In [31]:
# Output Census Tracts to Get Images
lc = pd.read_csv(survey_dir+"location.csv")
lc['geoid'] = lc['state_fips'].astype(str)+"_"+lc['county_fips'].astype(str)+"_"+lc['tract_fips'].astype(str)

lc = lc[['geoid','state_fips','county_fips','tract_fips','latitude','longitude']].drop_duplicates()

In [34]:
# 1. By OD pairs (741 tracts used on models before 20220208)
#tracts = pd.concat([od['tract_1'], od['tract_2']]).drop_duplicates().tolist()

# 2. By Trip Origins (1337 tracts)
# tracts = o_mode[o_mode['sampno_all']>=15]['geoid'].drop_duplicates().tolist()

# 3. By Trip Origins (1571 tracts)
tracts = o_mode[o_mode['sampno_all']>=10]['geoid'].drop_duplicates().tolist()

lc = lc[lc['geoid'].isin(tracts)]

In [35]:
# To make sure the train/test set is consistent with previous runs
backup = pd.read_csv(data_dir+"census_tracts_filtered-1337.csv")

lc = pd.merge(lc, backup[['geoid','train_test']], on='geoid', how='left')

In [36]:
# Assign train/test status to newly added tracts
lc['train_test'] = [int(np.random.rand() < 0.1) if np.isnan(x) else x for x in lc['train_test']]

In [37]:
lc.to_csv(data_dir+"census_tracts_filtered.csv", index=False)
print("# Census tracts in filtered OD pairs:", len(tracts))

# Census tracts in filtered OD pairs: 1571
